In [1]:
import pandas as pd
paris_df = pd.read_csv("data/noe_airbnb_dataset.csv")

In [2]:
def to_percentage(price, cleaning_fee):
    weight = (cleaning_fee/(price+cleaning_fee))*100
    return str(weight)+ "%" #str transforms the float into string

In [3]:
to_percentage(100,50)

'33.33333333333333%'

In [4]:
paris_df.dropna(subset=["price"])

,listing_url,name,summary,space,description,experiences_offered,host_id,host_since,host_response_rate,host_neighbourhood,...,longitude,property_type,room_type,bathrooms,bedrooms,price,cleaning_fee,minimum_nights,review_scores_rating,reviews_per_month
0,https://www.airbnb.com/rooms/2577,Loft for 4 by Canal Saint Martin,"100 m2 loft (1100 sq feet) with high ceiling, ...",The district has any service or shop you may d...,"100 m2 loft (1100 sq feet) with high ceiling, ...",none,2827,2008-09-09,75%,République,...,2.362511,Loft,Entire home/apt,2.0,2.0,125,40,3,NaN,NaN
1,https://www.airbnb.com/rooms/3109,zen and calm,Appartement très calme de 50M2 Belle lumière D...,I bedroom appartment in Paris 14,I bedroom appartment in Paris 14,none,3631,2008-10-14,100%,Alésia,...,2.318518,Apartment,Entire home/apt,1.0,1.0,75,50,3,100.0,0.29
2,https://www.airbnb.com/rooms/5396,Explore the heart of old Paris,"Cozy, well-appointed and graciously designed s...","Small, well appointed studio apartment at the ...","Cozy, well-appointed and graciously designed s...",none,7903,2009-02-14,100%,Saint-Paul - Ile Saint-Louis,...,2.358690,Apartment,Entire home/apt,1.0,0.0,115,36,2,94.0,1.29
3,https://www.airbnb.com/rooms/7397,MARAIS - 2ROOMS APT - 2/4 PEOPLE,"VERY CONVENIENT, WITH THE BEST LOCATION !",PLEASE ASK ME BEFORE TO MAKE A REQUEST !!! No ...,"VERY CONVENIENT, WITH THE BEST LOCATION ! PLEA...",none,2626,2008-08-30,100%,Le Marais,...,2.352751,Apartment,Entire home/apt,1.0,2.0,115,50,10,94.0,2.47
4,https://www.airbnb.com/rooms/7964,Large & sunny flat with balcony !,Very large & nice apartment all for you! - Su...,hello ! We have a great 75 square meter apartm...,Very large & nice apartment all for you! - Su...,none,22155,2009-06-18,75%,Gare du Nord - Gare de I'Est,...,2.343411,Apartment,Entire home/apt,1.0,2.0,99,60,3,96.0,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59065,https://www.airbnb.com/rooms/30621199,Charming and Calme Studio for two Persons,20m2 studio near avenue Foch in a very quiet s...,Studio de 20m2 à proximité de l'avenue Foch da...,20m2 studio near avenue Foch in a very quiet s...,none,44511985,2015-09-18,100%,Châtelet - Les Halles - Beaubourg,...,2.281544,Serviced apartment,Entire home/apt,1.0,0.0,45,50,30,NaN,NaN
59066,https://www.airbnb.com/rooms/30627500,Studio récent 28m2 au bord de Paris,Charmant studio de 28m2 dans un quartier neuf ...,NaN,Charmant studio de 28m2 dans un quartier neuf ...,none,153886633,2017-10-09,NaN,NaN,...,2.330878,Apartment,Entire home/apt,1.0,0.0,40,0,1,NaN,1.00
59067,https://www.airbnb.com/rooms/30630615,"Studio charme, Canal St Martin",Studio chaleureux situé sur le canal St Martin...,NaN,Studio chaleureux situé sur le canal St Martin...,none,25351458,2014-12-29,NaN,Canal Saint-Martin,...,2.366238,Apartment,Entire home/apt,1.0,0.0,58,0,2,NaN,NaN
59068,https://www.airbnb.com/rooms/30637928,LUXURY APARTMENT - Etoile / Arc de Triomphe,"Luxury fully-renovated Apartment, in a beauti...",At home in Paris... An incredible hideaway if ...,"Luxury fully-renovated Apartment, in a beauti...",none,229551435,2018-12-07,NaN,Champs-Elysées,...,2.289126,Apartment,Entire home/apt,1.0,1.0,81,60,2,NaN,NaN


In [5]:
paris_df_2 = paris_df[paris_df.price != 0]
paris_df_2["lol"] = paris_df_2.apply(lambda row: to_percentage(row["price"],row["cleaning_fee"]), axis=1)

/Users/ninoraymond/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import pprint
import time
import typing

import requests


def is_error_response(http_response, seconds_to_sleep: float = 1) -> bool:
    """
    Returns False if status_code is 503 (system unavailable) or 200 (success),
    otherwise it will return True (failed). This function should be used
    after calling the commands requests.post() and requests.get().

    :param http_response:
        The response object returned from requests.post or requests.get.
    :param seconds_to_sleep:
        The sleep time used if the status_code is 503. This is used to not
        overwhelm the service since it is unavailable.
    """
    if http_response.status_code == 503:
        time.sleep(seconds_to_sleep)
        return False

    return http_response.status_code != 200


def get_json(url) -> typing.Union[dict, None]:
    """
    Returns json response if any. Returns None if no json found.

    :param url:
        The url go get the json from.
    """
    response = requests.get(url)
    if is_error_response(response):
        return None
    json_response = response.json()
    return json_response


def get_reviews(app_id, page=1) -> typing.List[dict]:
    """
    Returns a list of dictionaries with each dictionary being one review. 
    
    :param app_id:
        The app_id you are searching. 
    :param page:
        The page id to start the loop. Once it reaches the final page + 1, the 
        app will return a non valid json, thus it will exit with the current 
        reviews. 
    """
    print(f'STARTED {page}')
    reviews: typing.List[dict] = [{}]

    while True:
        url = (f'https://itunes.apple.com/fr/rss/customerreviews/page={page}/id={app_id}/sortBy=mostRecent/json')
        json = get_json(url)

        if not json:
            return reviews

        data_feed = json.get('feed')

        try:
            if not data_feed.get('entry'):
                get_reviews(app_id, page + 1)


            reviews += [
                {
                    'review_id': entry.get('id').get('label'),
                    'title': entry.get('title').get('label'),
                    'author': entry.get('author').get('name').get('label'),
                    'author_url': entry.get('author').get('uri').get('label'),
                    'version': entry.get('im:version').get('label'),
                    'rating': entry.get('im:rating').get('label'),
                    'review': entry.get('content').get('label'),
                    'vote_count': entry.get('im:voteCount').get('label'),
                    'page': page
                }
                for entry in data_feed.get('entry')
                if not entry.get('im:name')
            ]
            page += 1
        except Exception:
            return reviews

In [ ]:
reviews = get_reviews("925339063")

In [ ]:
doctolib_df = pd.DataFrame(columns=['review', 'rating', 'version'])

In [ ]:
df = pd.DataFrame(reviews)

In [ ]:
df

In [ ]:
from translate import Translator
translator= Translator(to_lang="fr")
translation = translator.translate("This is a pen.")

In [ ]:
#lol = df.apply(lambda row: ts.google(row["review"]),axis=1)

In [7]:
lol

NameError: name 'lol' is not defined

In [10]:
import translators as ts
ts.google("Au temps pour moi")

'My mistake'

In [ ]:
lol